In [16]:
import gc
import time
import pandas as pd
import datetime
import polars as pl
import numpy as np
import argparse
import yaml
import logging
import torch
from torch.utils.data import Dataset, DataLoader

In [28]:
import json
import os

def activate_kaggle_user(user):
    # 사용자 키가 저장된 파일 경로
    filepath = '/home/jhko/LEAP_ClimSim/kaggle_copy.json'

    # kaggle.json 파일 읽기
    with open(filepath, 'r') as f:
        users = json.load(f)
    
    if user not in users:
        raise ValueError(f"User {user} not found in the kaggle.json file")

    user_info = users[user]

    # kaggle.json 파일에 덮어쓰기
    with open('/home/jhko/LEAP_ClimSim/kaggle.json', 'w') as f:
        json.dump(user_info, f)

    # 파일 권한 설정
    os.chmod('/home/jhko/LEAP_ClimSim/kaggle.json', 0o600)
    print(f"Activated Kaggle user: {user}")

# 예시 사용법
activate_kaggle_user('limkim')


Activated Kaggle user: limkim


In [17]:
import random

def load_config(config_path):
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
    return config

def setup_logging(log_file):
    logging.basicConfig(level = logging.INFO, format='%(asctime)s - %(message)s', handlers = [
        logging.FileHandler(log_file, mode = 'a'),
        logging.StreamHandler()
    ])
    
def log_hyperparameters(config):
    for key, value in config.items():
        logging.info(f"{key}: {value}")

def format_time(elapsed):
    """Take a time in seconds and return a string hh:mm:ss."""
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

def seed_everything(seed_val=1325):
    """Seed everything."""
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

In [18]:
def load_train():

    weights = pd.read_csv('/data01/jhko/LEAP/sample_submission.csv', nrows = 1)
    del weights['sample_id']
    weights = weights.T
    weights = weights.to_dict()[0]

    df_train = pl.read_csv('/data01/jhko/LEAP/train.csv', n_rows = 2_500_500)
    for target in weights:
        df_train = df_train.with_columns(pl.col(target).mul(weights[target]))
    print('time to read dataset:', format_time(time.time()-ts), flush = True)

    FEAT_COLS = df_train.columns[1:557]
    TARGET_COLS = df_train.columns[557:]

    for col in FEAT_COLS:
        df_train = df_train.with_columns(pl.col(col).cast(pl.Float32))

    for col in TARGET_COLS:
        df_train = df_train.with_columns(pl.col(col).cast(pl.Float32))

    x_train = df_train.select(FEAT_COLS).to_numpy()
    y_train = df_train.select(TARGET_COLS).to_numpy()

    del df_train
    gc.collect()
    
    return x_train, y_train, FEAT_COLS, TARGET_COLS

def normalization(x_train, y_train):
    mx = x_train.mean(axis=0)
    sx = np.maximum(x_train.std(axis=0), 1e-8)
    x_train = (x_train - mx.reshape(1,-1)) / sx.reshape(1,-1)

    my = y_train.mean(axis=0)
    sy = np.maximum(np.sqrt((y_train*y_train).mean(axis=0)), 1e-8)
    y_train = (y_train - my.reshape(1,-1)) / sy.reshape(1,-1)

    return x_train, y_train, mx, sx, my, sy

In [19]:
class NumpyDataset(Dataset):
    def __init__(self, x, y):
        assert x.shape[0] == y.shape[0], "x, y sample shape not same"
        self.x = x
        self.y = y
        
    def __len__(self):
        return self.x.shape[0]
    
    def __getitem__(self, index):
        return torch.from_numpy(self.x[index]).float().to(device), torch.from_numpy(self.y[index]).float().to(device)
        

In [20]:
import torch.nn as nn

class FFNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(FFNN, self).__init__()
        
        layers = []
        previous_size = input_size
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(previous_size, hidden_size))
            layers.append(nn.LayerNorm(hidden_size))
            layers.append(nn.LeakyReLU(inplace=True))
            layers.append(nn.Dropout(p=0.1))
            previous_size = hidden_size
        
        layers.append(nn.Linear(previous_size, output_size))
        
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

In [21]:
class LpLoss(object):
    def __init__(self, d=2, p=2, size_average=True, reduction=True):
        super(LpLoss, self).__init__()

        assert d>0 and p>0

        self.d = d
        self.p = p
        self.reduction = reduction
        self.size_average = size_average

    def abs(self, x, y):
        num_examples = x.size()[0]
        h = 1.0 / (x.size()[1] - 1.0)

        all_norms = (h**(self.d/self.p)) * torch.norm(x.view(num_examples, -1) - y.view(num_examples, -1), self.p, 1)
        if self.reduction:
            if self.size_average:
                return torch.mean(all_norms)
            else:
                return torch.sum(all_norms)

        return all_norms

    def rel(self, x, y):
        epsilon = 1e-8
        num_examples = x.size()[0]

        diff_norms = torch.norm(x.reshape(num_examples, -1) - y.reshape(num_examples, -1), self.p, 1)
        y_norms = torch.norm(y.reshape(num_examples, -1), self.p, 1) + epsilon

        if self.reduction:
            if self.size_average:
                return torch.mean(diff_norms / y_norms)
            else:
                return torch.sum(diff_norms / y_norms)
        return diff_norms / y_norms

    def __call__(self, x, y):
        return self.rel(x, y)

In [22]:
import torch.optim as optim
from torchmetrics.regression import R2Score

if __name__ == "__main__":
    
    parser = argparse.ArgumentParser(description='Main script for training the model.')
    parser.add_argument('--config', type=str, default='hyper.yaml', help='Path to the config file..')
    args, unknown = parser.parse_known_args()

    config = load_config(args.config)
    setup_logging('training.log')
    
    logging.info("-----------------------------------------------------------------------------")
    logging.info(f"Hyperparameters:")
    log_hyperparameters(config)

    ts = time.time()
    
    x_train, y_train, FEAT_COLS, TARGET_COLS = load_train() 
    x_train, y_train, mx, sx, my, sy = normalization(x_train, y_train) # x_train:[_, 556(feat_cols)], y_train:[_, 368(target_cols)]
    
    seed_everything()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("am i using gpu? :", device)
    
    dataset = NumpyDataset(x_train, y_train)
    train_size = int(0.9 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
    train_loader = DataLoader(train_dataset, batch_size = config['BATCH_SIZE'] , shuffle = True)
    val_loader = DataLoader(val_dataset, batch_size = config['BATCH_SIZE'], shuffle=False)
    
    input_size = x_train.shape[1] #556
    output_size = y_train.shape[1] #368
    hidden_size = input_size + output_size #556+368
    model = FFNN(input_size, [3*hidden_size, 2*hidden_size, 2*hidden_size, 2*hidden_size, 3*hidden_size], output_size).to(device)
    criterion = LpLoss()
    optimizer = optim.AdamW(model.parameters(), lr = config['LEARNING_RATE'], weight_decay = config['WEIGHT_DECAY'])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=config['SCHEDULER_FACTOR'], patience=config['SCHEDULER_PATIENCE'])
    print("Time after all preparations:", format_time(time.time()-ts), flush=True)
    
    best_val_loss = float('inf')
    best_model_state = None
    patience_count = 0
    r2score = R2Score(num_outputs=len(TARGET_COLS)).to(device)
    
    for epoch in range(config['EPOCHS']):
        print(" ")
        model.train()
        total_loss = 0
        steps = 0
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels) #EDA
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            steps += 1

            if (batch_idx + 1) % config['PRINT_FREQ'] == 0 :
                current_lr = optimizer.param_groups[0]['lr']
                elapsed_time = format_time(time.time() - ts)
                log_message = (
                              f'  Epoch: {epoch+1}',\
                              f'  Batch: {batch_idx + 1}/{len(train_loader)}',\
                              f'  Train Loss: {total_loss / steps:.4f}',\
                              f'  LR: {current_lr:.1e}',\
                              f'  Time: {elapsed_time}'
                              )
                logging.info(log_message)
                total_loss = 0
                steps = 0

        model.eval()
        val_loss = 0
        y_true = torch.tensor([], device = device)
        all_outputs = torch.tensor([], device = device)
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                val_loss += criterion(outputs, labels).item()
                y_true = torch.cat((y_true, labels), 0)
                all_outputs = torch.cat((all_outputs, outputs), 0) #vs. train EDA
        r2 = 0
        r2_broken = []
        r2_broken_names = []
        for i in range(output_size): # 368
            r2_i = r2score(all_outputs[:, i], y_true[:, i])
            if r2_i > 1e-6:
                r2 += r2_i
            else:
                r2_broken.append(i)
                r2_broken_names.append(FEAT_COLS[i])
        r2 /= output_size

        avg_val_loss = val_loss / len(val_loader)
        val_log = (f'\nEpoch: {epoch+1}  Val Loss: {avg_val_loss:.4f}  R2 score: {r2:.4f}')
        logging.info(val_log)

        print(f'{len(r2_broken)} targets were excluded during evaluation of R2 score.')
        #print(f"r2_broken:, {r2_broken}")
        #print(f"r2_broken_names: {r2_broken_names}")

        scheduler.step(avg_val_loss)

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict()
            patience_count = 0
            log_msg = ("Validation loss decreased, saving new best model and resetting patience counter.")
            logging.info(log_msg)
        else:
            patience_count += 1
            log_msg = (f"No improvement in validation loss for {patience_count} epochs.")
            logging.info(log_msg)

        if patience_count >= config['PATIENCE']:
            log_msg = ("Stopping early due to no improvement in validation loss.")
            logging.info(log_msg)
            break

del x_train, y_train
gc.collect()

model.load_state_dict(best_model_state)
model.eval()

df_test = pl.read_csv('/data01/jhko/LEAP/test.csv')

for col in FEAT_COLS:
    df_test = df_test.with_columns(pl.col(col).cast(pl.Float32))
    
x_test = df_test.select(FEAT_COLS).to_numpy()
x_test = (x_test - mx.reshape(1,-1)) / sx.reshape(1,-1)
predt = np.zeros([x_test.shape[0], output_size], dtype=np.float32)

i1 = 0
for i in range(10000):
    i2 = np.minimum(i1 + config['BATCH_SIZE'], x_test.shape[0])
    if i1 == i2:
        break
        
    inputs = torch.from_numpy(x_test[i1:i2, :]).float().to(device)
    
    with torch.no_grad():
        outputs = model(inputs)
        predt[i1:i2, :] = outputs.cpu().numpy()
        
    i1 = i2
    
    if i2 >= x_test.shape[0]:
        break
        
for i in range(sy.shape[0]):
    if sy[i] < 1e-8 * 1.1:
        predt[:, i] = 0
        
predt = predt * sy.reshape(1, -1) + my.reshape(1, -1)

ss = pd.read_csv("/data01/jhko/LEAP/sample_submission.csv")
ss.iloc[:, 1:] = ss.iloc[:, 1:].astype('float')
ss.iloc[:, 1:] = predt

del predt
gc.collect()

use_cols = []
for i in range(27):
    use_cols.append(f"ptend_q0002_{i}")

ss2 = pd.read_csv("/data01/jhko/LEAP/sample_submission.csv")
df_test = df_test.to_pandas()

for col in use_cols:
    ss[col] = -df_test[col.replace("ptend", "state")]*ss2[col]/1200.

test_polars = pl.from_pandas(ss[["sample_id"]+TARGET_COLS])
test_polars.write_csv("submission.csv")

print("Total time:", format_time(time.time()-ts))

2024-06-24 13:59:07,884 - -----------------------------------------------------------------------------
2024-06-24 13:59:07,885 - Hyperparameters:
2024-06-24 13:59:07,886 - DATA_PATH: /data01/jhko/LEAP/
2024-06-24 13:59:07,886 - BATCH_SIZE: 1024
2024-06-24 13:59:07,886 - MIN_STD: 1e-8
2024-06-24 13:59:07,887 - SCHEDULER_PATIENCE: 3
2024-06-24 13:59:07,887 - SCHEDULER_FACTOR: 0.316
2024-06-24 13:59:07,887 - EPOCHS: 10
2024-06-24 13:59:07,888 - PATIENCE: 6
2024-06-24 13:59:07,888 - PRINT_FREQ: 50
2024-06-24 13:59:07,888 - BEST_MODEL_PATH: best_model_seq.pth
2024-06-24 13:59:07,889 - LEARNING_RATE: 0.001
2024-06-24 13:59:07,889 - WEIGHT_DECAY: 0.01


time to read dataset: 0:00:36
am i using gpu? : cuda:0
Time after all preparations: 0:01:32
 


2024-06-24 14:00:46,626 - ('  Epoch: 1', '  Batch: 50/2198', '  Train Loss: 1.1479', '  LR: 1.0e-03', '  Time: 0:01:39')
2024-06-24 14:00:51,809 - ('  Epoch: 1', '  Batch: 100/2198', '  Train Loss: 0.9956', '  LR: 1.0e-03', '  Time: 0:01:44')
2024-06-24 14:00:56,766 - ('  Epoch: 1', '  Batch: 150/2198', '  Train Loss: 0.8866', '  LR: 1.0e-03', '  Time: 0:01:49')
2024-06-24 14:01:01,747 - ('  Epoch: 1', '  Batch: 200/2198', '  Train Loss: 0.8625', '  LR: 1.0e-03', '  Time: 0:01:54')
2024-06-24 14:01:06,808 - ('  Epoch: 1', '  Batch: 250/2198', '  Train Loss: 0.8284', '  LR: 1.0e-03', '  Time: 0:01:59')
2024-06-24 14:01:11,859 - ('  Epoch: 1', '  Batch: 300/2198', '  Train Loss: 0.8058', '  LR: 1.0e-03', '  Time: 0:02:04')
2024-06-24 14:01:16,990 - ('  Epoch: 1', '  Batch: 350/2198', '  Train Loss: 0.7851', '  LR: 1.0e-03', '  Time: 0:02:09')
2024-06-24 14:01:21,974 - ('  Epoch: 1', '  Batch: 400/2198', '  Train Loss: 0.7730', '  LR: 1.0e-03', '  Time: 0:02:14')
2024-06-24 14:01:27,389 -

93 targets were excluded during evaluation of R2 score.
 


2024-06-24 14:04:53,546 - ('  Epoch: 2', '  Batch: 50/2198', '  Train Loss: 0.6440', '  LR: 1.0e-03', '  Time: 0:05:46')
2024-06-24 14:04:58,764 - ('  Epoch: 2', '  Batch: 100/2198', '  Train Loss: 0.6421', '  LR: 1.0e-03', '  Time: 0:05:51')
2024-06-24 14:05:04,170 - ('  Epoch: 2', '  Batch: 150/2198', '  Train Loss: 0.6411', '  LR: 1.0e-03', '  Time: 0:05:56')
2024-06-24 14:05:09,154 - ('  Epoch: 2', '  Batch: 200/2198', '  Train Loss: 0.6412', '  LR: 1.0e-03', '  Time: 0:06:01')
2024-06-24 14:05:14,083 - ('  Epoch: 2', '  Batch: 250/2198', '  Train Loss: 0.6365', '  LR: 1.0e-03', '  Time: 0:06:06')
2024-06-24 14:05:19,018 - ('  Epoch: 2', '  Batch: 300/2198', '  Train Loss: 0.6387', '  LR: 1.0e-03', '  Time: 0:06:11')
2024-06-24 14:05:23,956 - ('  Epoch: 2', '  Batch: 350/2198', '  Train Loss: 0.6381', '  LR: 1.0e-03', '  Time: 0:06:16')
2024-06-24 14:05:29,004 - ('  Epoch: 2', '  Batch: 400/2198', '  Train Loss: 0.6380', '  LR: 1.0e-03', '  Time: 0:06:21')
2024-06-24 14:05:33,940 -

...

90 targets were excluded during evaluation of R2 score.
 


2024-06-24 14:30:45,255 - ('  Epoch: 9', '  Batch: 50/2198', '  Train Loss: 0.5614', '  LR: 1.0e-03', '  Time: 0:31:37')
2024-06-24 14:30:49,620 - ('  Epoch: 9', '  Batch: 100/2198', '  Train Loss: 0.5585', '  LR: 1.0e-03', '  Time: 0:31:42')
2024-06-24 14:30:54,016 - ('  Epoch: 9', '  Batch: 150/2198', '  Train Loss: 0.5602', '  LR: 1.0e-03', '  Time: 0:31:46')
2024-06-24 14:30:58,522 - ('  Epoch: 9', '  Batch: 200/2198', '  Train Loss: 0.5595', '  LR: 1.0e-03', '  Time: 0:31:51')
2024-06-24 14:31:03,057 - ('  Epoch: 9', '  Batch: 250/2198', '  Train Loss: 0.5597', '  LR: 1.0e-03', '  Time: 0:31:55')
2024-06-24 14:31:07,581 - ('  Epoch: 9', '  Batch: 300/2198', '  Train Loss: 0.5608', '  LR: 1.0e-03', '  Time: 0:32:00')
2024-06-24 14:31:11,856 - ('  Epoch: 9', '  Batch: 350/2198', '  Train Loss: 0.5611', '  LR: 1.0e-03', '  Time: 0:32:04')
2024-06-24 14:31:16,219 - ('  Epoch: 9', '  Batch: 400/2198', '  Train Loss: 0.5613', '  LR: 1.0e-03', '  Time: 0:32:08')
2024-06-24 14:31:20,473 -

89 targets were excluded during evaluation of R2 score.
 


2024-06-24 14:34:17,386 - ('  Epoch: 10', '  Batch: 50/2198', '  Train Loss: 0.5571', '  LR: 1.0e-03', '  Time: 0:35:09')
2024-06-24 14:34:21,764 - ('  Epoch: 10', '  Batch: 100/2198', '  Train Loss: 0.5552', '  LR: 1.0e-03', '  Time: 0:35:14')
2024-06-24 14:34:26,003 - ('  Epoch: 10', '  Batch: 150/2198', '  Train Loss: 0.5574', '  LR: 1.0e-03', '  Time: 0:35:18')
2024-06-24 14:34:30,259 - ('  Epoch: 10', '  Batch: 200/2198', '  Train Loss: 0.5558', '  LR: 1.0e-03', '  Time: 0:35:22')
2024-06-24 14:34:34,731 - ('  Epoch: 10', '  Batch: 250/2198', '  Train Loss: 0.5575', '  LR: 1.0e-03', '  Time: 0:35:27')
2024-06-24 14:34:39,216 - ('  Epoch: 10', '  Batch: 300/2198', '  Train Loss: 0.5572', '  LR: 1.0e-03', '  Time: 0:35:31')
2024-06-24 14:34:43,844 - ('  Epoch: 10', '  Batch: 350/2198', '  Train Loss: 0.5561', '  LR: 1.0e-03', '  Time: 0:35:36')
2024-06-24 14:34:48,312 - ('  Epoch: 10', '  Batch: 400/2198', '  Train Loss: 0.5547', '  LR: 1.0e-03', '  Time: 0:35:40')
2024-06-24 14:34:

89 targets were excluded during evaluation of R2 score.
Total time: 0:39:22
